### 1. Loading Dataset
- I have taked data of audio from website : https://commonvoice.mozilla.org/en/datasets and version : Common Voice Delta Segment 19.0
- downloaded it -> unzip it and saved it in -> data/all_audio_data 
- downloaded data has a file "validated.tsv" that contains 2 imp. data 'client_id' , 'path(file name)' of the audio and all other metadata

In [66]:
import pandas as pd

In [67]:
# loading validated.tsv into a pandas dataframe
df = pd.read_csv(r"C:\Users\shubu\Desktop\Speaker Verification System\data\validated.tsv", sep="\t")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   client_id        137 non-null    object 
 1   path             137 non-null    object 
 2   sentence_id      137 non-null    object 
 3   sentence         137 non-null    object 
 4   sentence_domain  1 non-null      object 
 5   up_votes         137 non-null    int64  
 6   down_votes       137 non-null    int64  
 7   age              117 non-null    object 
 8   gender           91 non-null     object 
 9   accents          136 non-null    object 
 10  variant          0 non-null      float64
 11  locale           137 non-null    object 
 12  segment          0 non-null      float64
dtypes: float64(2), int64(2), object(9)
memory usage: 14.0+ KB


- since we need some target data and non target data to train out model 
- so we find a client_id with most numbers of data/audio to make it a reference audio to train model on

In [68]:
counts = df["client_id"].value_counts() # finds the number of rows of client id

print(counts.head(5)) # Print top 5 most-frequent client id values

client_id
b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467d52e816db909dbe2be05b8ea36a4bca91179503817a1b093a9e6036abb514c6ad1aaf1ffa71b3d32ce    28
60420cbff86f4f0a421e0f797a5b263ed86e12b0d504f47f0f2cfafd2728b30364575c58f6194feb4c777221e426723c9c70b52f267c0d52ae64bcd61d3ee9cf    20
64682a2154ab6c1f1179cca7deb92457947174c2f45db0f3f30acfac052f98ac1ef71f780a721c76f86e958b35cae92296cb7bf0c2414ea73342a00a48854942    15
e8e299ceb26b0e13b890a83f6e2aa8c464520fe1271a4ab4f4e853f296d4c70c50f829f7bd0eae70d82b1f9b93f432c70682122db4fb7427c802ff11ceba0fd5     8
6fb7fe81a15e5e14ea771b047a81a786203287eabe7d4f2d616a2f51d329704516c327d7a30c761d38016d708da9365466cab653ea6d02ab16c47def9936f1ef     8
Name: count, dtype: int64


- we see that client_id = b87.. has 28 audios wich can be used as training data for target speaker(reference audio)
- so now we make a new data frame for the 'target speaker' values

In [69]:
#  we choose client id with 28 clips : we will use this as target speaker training data
target_id = "b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467d52e816db909dbe2be05b8ea36a4bca91179503817a1b093a9e6036abb514c6ad1aaf1ffa71b3d32ce"

# makiing new data frame
df_target = df[df["client_id"] == target_id].copy()

print(f"no. of target clips: {len(df_target)}") 
df_target.head()


no. of target clips: 28


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
109,b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467...,common_voice_en_40882518.mp3,ee9b750a4e08de54ce5742ba8bbeaf8209a10a9f5cc882...,The district had its origins in the Ware Rural...,NaN,2,0,thirties,male_masculine,Canadian English,NaN,en,NaN
110,b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467...,common_voice_en_40903624.mp3,eec76b5d373cada19063f29ab4d9ee73988d6fef4ec7e5...,Yet fortune would soon turn on the Grey househ...,NaN,2,0,thirties,male_masculine,Canadian English,NaN,en,NaN
111,b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467...,common_voice_en_40903655.mp3,eec10bab785aa378caa2cfe5feb123a00a372b98bf527c...,He was an enigmatic figure for most of his car...,NaN,2,0,thirties,male_masculine,Canadian English,NaN,en,NaN
112,b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467...,common_voice_en_40923835.mp3,eee3001d454b935f22a8efa974b84b128e1dfbea29e3e0...,Eyes are small.,NaN,2,0,thirties,male_masculine,Canadian English,NaN,en,NaN
113,b87dd0680063cbd3dc57f5b9b20c14f020f511c3f18467...,common_voice_en_40924027.mp3,ef1bd2150466b7242c84ec70ae390b37267e4fb859a201...,"Lloyd was born in Glasgow, Scotland.",NaN,2,0,thirties,male_masculine,Canadian English,NaN,en,NaN


- we also need the data for 'non-target speaker' values for training so, we randomly select 40 rows from the 'df' and make a new dataframe  'df_non_target'

In [70]:
# all data except the target speaker data
df_non_target_all = df[df["client_id"] != target_id].copy()

# randomly selecting 40 audios 
df_non_target = df_non_target_all.sample(n=40, random_state=42).reset_index(drop=True)

print(f"Number of non-target clips: {len(df_non_target)}")
df_non_target.head()

Number of non-target clips: 40


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,64682a2154ab6c1f1179cca7deb92457947174c2f45db0...,common_voice_en_41113888.mp3,f0c37c28e0761e07163b8c9893ed039c789de7f4feba64...,"As the popularity of bridging loans increased,...",NaN,2,0,twenties,female_feminine,United States English,NaN,en,NaN
1,b895602d9eae132fe68b85ff4a1af9d377f0f324686c25...,common_voice_en_41069015.mp3,f07b5c4c8d545f5d76281f14b1dbad58e73f637ca1d6ad...,The historicity of Samyogita is a matter of de...,NaN,2,0,twenties,NaN,United States English,NaN,en,NaN
2,38a6ab913e27c72c3f28f3d2da65caa00933288bfaaa24...,common_voice_en_40983032.mp3,efe4e03f95b62047b25b5cadab7793fd1e517ea9fbac73...,"On Sundays, four trains run each way, to Carli...",NaN,2,0,twenties,NaN,"India and South Asia (India, Pakistan, Sri Lanka)",NaN,en,NaN
3,64682a2154ab6c1f1179cca7deb92457947174c2f45db0...,common_voice_en_41118674.mp3,f0f4a3700c9a630321b7cc89f8c9265f2e827d09eaa801...,He was a university student in both cities and...,NaN,2,0,twenties,female_feminine,United States English,NaN,en,NaN
4,6fb7fe81a15e5e14ea771b047a81a786203287eabe7d4f...,common_voice_en_40960881.mp3,efa1cddc7d5e9e6c6673b052376d371d7469ec42386ea1...,Both the blue and white jerseys featured red s...,NaN,2,0,NaN,NaN,United States English,NaN,en,NaN


- now, we have 2 data frame 'df_target' and 'df_not_target' which contain all the required path for audios that will be used for training the ML model
- both data frame has a coulumn 'path' but it has just file name. 
- so, we will extract those file names and create new column 'full_path' by adding the file path and file name

In [71]:
import os

CLIPS_DIRECTORY = r"C:\Users\shubu\Desktop\Speaker Verification System\data\clips" # folder path with all the audios

# creating new column “full_path” with directory and file name : absolute path
df_target["full_path"] = df_target["path"].apply(lambda filename: os.path.join(CLIPS_DIRECTORY, filename))
df_non_target["full_path"] = df_non_target["path"].apply(lambda filename: os.path.join(CLIPS_DIRECTORY, filename))

# df_target["full_path"].head(), df_non_target["full_path"].head() 
df_target["full_path"].head()

109    C:\Users\shubu\Desktop\Speaker Verification Sy...
110    C:\Users\shubu\Desktop\Speaker Verification Sy...
111    C:\Users\shubu\Desktop\Speaker Verification Sy...
112    C:\Users\shubu\Desktop\Speaker Verification Sy...
113    C:\Users\shubu\Desktop\Speaker Verification Sy...
Name: full_path, dtype: object

### 2. for finding MFCC statistics from each audio file
- for each audio clip:
- load at 16 kHz.
- Trim silence
- Compute 13 MFCCs (shape = (13, T) for some T frames).
- Take the mean and standard deviation of each of the 13 coefficients → that gives us a 26-dim vector (13 means + 13 stds).


#### MFCC Statistics detail
- Seeing sound -> curvy path on paper -> can be thousands of numbers long -> we squeeze all that info. into just few important numbers -> MFCCs help us find those important numbers

- Mel‐Frequency Cepstral Coefficients -> a special set of measurements -> We pick 13 of these special measurements -> for every short piece of the sound, we get 13 numbers.

- we split the sound into little pieces -> For each tiny piece, we find those 13 MFCC numbers.

- we have those 13 numbers for every little chunk -> 100 chunks for a 2‐second audio -> big table -> we don’t keep all 100 rows. Instead, we take average (mean) of these 100 numbers

- How much do they jump up and down -> standard deviation

- Since there are 13 MFCC measurements, we get 13 “means” and 13 “standard deviations” - > These 26 numbers are the “MFCC statistics” for that entire audio clip

In [72]:
# function to find MFCC statistics
import numpy as np
import librosa

# Configuration
SAMPLE_RATE = 16000
N_MFCC = 13

def MFCC_STATISTICS(filepath, sr=SAMPLE_RATE, n_mfcc=N_MFCC):
    
    wav, _ = librosa.load(filepath, sr=sr)   # load the file at 16 kHz
    wav, _ = librosa.effects.trim(wav)       # silence removed
    mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=n_mfcc)
    means = np.mean(mfcc, axis=1)                       
    stds  = np.std(mfcc, axis=1)                        
    return np.concatenate([means, stds])                


- now, i loop over all 28 target clips, then all 40 non-target clips
- extract their 26-dim feature vectors, and store them in x and make label y (y = 1 for each target clip, y = 0 for each non-target clip)

In [73]:
x = []
y = []

# target clips (label = 1)
for fp in df_target["full_path"]:
    vec = MFCC_STATISTICS(fp)
    x.append(vec)
    y.append(1)

# non-target clips (label = 0)
for fp in df_non_target["full_path"]:
    vec = MFCC_STATISTICS(fp)
    x.append(vec)
    y.append(0)

# convert list to np array
x = np.vstack(x)   # shape = (28 + 40, 26)
y = np.array(y)    # shape = (68,)
print("Features :", x.shape)
print("Label :", y.shape)


Features : (68, 26)
Label : (68,)


### 3. Train_Test_Split
- now, we have x and y (input features and output feature data)
- so, now we split out data for training and testing

In [74]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x , y , test_size= 0.2, random_state=1, stratify=y)

print("x_train:", x_train.shape, "y_train:", y_train.shape)
print("x_test: ", x_test.shape, "y_test: ", y_test.shape)

x_train: (54, 26) y_train: (54,)
x_test:  (14, 26) y_test:  (14,)


### 4. Model
##### We used SVC (Support Vector Classifier): 
- SVC works well on smaller datasets
- after extracting MFCC stats, the data becomes high dimentional
- SVC handles high-dimensional data and finds an optimal decision boundary

In [75]:
from sklearn.svm import SVC

clf = SVC(kernel="rbf", probability=True, random_state=2)
clf.fit(x_train, y_train)

SVC(probability=True, random_state=2)

### 5. Evaluation

In [76]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

y_pred = clf.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
f1_score  = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1_score:.4f}\n")
print("report:\n")
print(classification_report(y_test, y_pred, target_names=["non-target","target"]))

Accuracy: 0.7857
F1-score: 0.7692

report:

              precision    recall  f1-score   support

  non-target       0.86      0.75      0.80         8
      target       0.71      0.83      0.77         6

    accuracy                           0.79        14
   macro avg       0.79      0.79      0.78        14
weighted avg       0.80      0.79      0.79        14



### 6. Model Extraction

In [77]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(clf, f)

### 7. Prediction

In [91]:
test_audio_path = r"C:\Users\shubu\Desktop\Speaker Verification System\testing_data\common_voice_en_40903624.mp3"  

# MFCC statistics 
features = MFCC_STATISTICS(test_audio_path)       
features = features.reshape(1, -1)                

# run trained SVC on this example
pred_label = clf.predict(features)[0]             
pred_probability  = clf.predict_proba(features)[0]        


label_map = {0: "non-target", 1: "target"}
pred_text = label_map[pred_label]

# result print
print(f"Predicted class: {pred_text}  (label = {pred_label})")
print(f"Confidence --> non-target: {pred_probability[0]:.4f},  target: {pred_probability[1]:.4f}")

Predicted class: target  (label = 1)
Confidence --> non-target: 0.0717,  target: 0.9283
